<a href="https://colab.research.google.com/github/dapopov-st/RagOverArXiv/blob/main/GenerateQaDatasetWithT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate QA Pairs with T5 for RAG Finetuning

In [3]:
!pip install ctransformers[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 59.5 MB/s eta 0:00:00


In [ ]:
!pip install lmqg

In [98]:
import os
from google.colab import drive
drive.mount('/content/drive/')

output_dir = '/content/drive/MyDrive/PdfRag/finetune_output_dir'
logging_dir = '/content/drive/MyDrive/PdfRag/finetune_logging_dir'
index_dir = '/content/drive/MyDrive/PdfRag/finetune_index_dir'
papers_dir = '/content/drive/MyDrive/PdfRag/clusterofstars'
data_dir = '/content/drive/MyDrive/PdfRag/data'
%cd /content/drive/MyDrive/PdfRag
!ls ./clusterofstars

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/PdfRag
'Atlas:  Few-shot Learning with Retrieval Augmented Language Models.pdf'
 ChainOfThought
'Chain-of-Thought Prompting Elicits Reasoning in Large Language Models.pdf'
'Chain-of-Verification Reduces Hallucination in Large Language Models.pdf'
'DSPy:  Compiling Declarative Language Model Calls into Self-Improving Pipelines.pdf'
'Evaluating Large Language Models Trained on Code.pdf'
 ExcludeSurveysAndLitReviews
'FlashAttention:  Fast and Memory-Efficient Exact Attention with IO-Awareness.pdf'
'From Sparse to Dense:  GPT-4 Summarization with Chain of Density Prompting.pdf'
'Graph of Thoughts: Solving Elaborate Problems with Large Language Models.pdf'
'In-Context Retrieval-Augmented Language Models.pdf'
'Least-to-Most Prompting Enables Complex Reasoning in Large Language Models.pdf'
'Leveraging Passage Retrieval with Generative Models

In [6]:
import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()
clean_mem()

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# MODEL_PATH = "TheBloke/Amethyst-13B-Mistral-GGUF"
# MODEL_FILE = 'amethyst-13b-mistral.Q6_K.gguf'
MODEL_PATH = "FPHam/Reverso_Expanded_13b_Q_Generator_GPTQ"
#MODEL_FILE = 'amethyst-13b-mistral.Q6_K.gguf'

In [9]:
!pip install ctransformers[gptq]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.1
    Uninstalling safetensors-0.4.1:
      Successfully uninstalled safetensors-0.4.1


In [10]:
import json

from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode

In [11]:
TRAIN_FILES = ['./clusterofstars/LoRA: Low-Rank Adaptation of Large Language Models.pdf']
VAL_FILES = ['./clusterofstars/QLORA: Efficient Finetuning of Quantized LLMs.pdf']

TRAIN_CORPUS_FPATH = './data/train_corpus.json'
VAL_CORPUS_FPATH = './data/val_corpus.json'

In [12]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.1 MB/s eta 0:00:00


In [44]:
from pdfminer.high_level import extract_text
import re
import pandas as pd
text = extract_text(TRAIN_FILES[0])

In [ ]:
#ORIGINAL from docs
from pdfminer.layout import LAParams
from pdfminer.converter import PDFResourceManager, PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LTTextBoxHorizontal
document = open(TRAIN_FILES[0], 'rb')
#Create resource manager
rsrcmgr = PDFResourceManager()
# Set parameters for analysis.
laparams = LAParams()
# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
for page in PDFPage.get_pages(document):
  interpreter.process_page(page)
  # receive the LTPage object for the page.
  layout = device.get_result()
  for element in layout:
    if isinstance(element, LTTextBoxHorizontal)
      print(element.get_text())

In [68]:
def mostly_numbers(text, threshold=0.5):
    # Find all numeric characters in the text
    numbers = re.findall(r'[\d()±/. ]', text)
    percentage = len(numbers) / len(text) if text else 1
    return percentage >= threshold
def starts_with_figure_or_table(text):
  pattern = r"^(Figure|Table)\s+\d+\s*:"
  return bool(re.match(pattern, text))

def ends_with_period_or_number(text):
    """...or dash"""
    pattern = r"[.\d-]$"
    return bool(re.search(pattern, text))

def join_hyphenated_words(text):
    pattern = r"([a-z]+)-\n\s*([a-z]+)"
    return re.sub(pattern, r'\1\2', text)

def ignore_footnote(text):
    "Works OK, just want to trim what's carried over on the next line in footnote"
    pattern = r"\n\d[A-Z][^.]*\."
    return bool(re.search(pattern, text))


from pdfminer.layout import LAParams
from pdfminer.converter import PDFResourceManager, PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LTTextBoxHorizontal
def parse_paragraphs():
  df = pd.DataFrame(columns=['abstract'])
  document = open(TRAIN_FILES[0], 'rb')
  #Create resource manager
  rsrcmgr = PDFResourceManager()
  # Set parameters for analysis.
  laparams = LAParams()
  # Create a PDF page aggregator object.
  device = PDFPageAggregator(rsrcmgr, laparams=laparams)
  interpreter = PDFPageInterpreter(rsrcmgr, device)
  SEEN_ABSTRACT = False
  buffer = ''
  for page in PDFPage.get_pages(document):
    interpreter.process_page(page)
    # receive the LTPage object for the page.
    layout = device.get_result()

    for element in layout:
      if isinstance(element, LTTextBoxHorizontal):
        text = element.get_text()
        text = join_hyphenated_words(text)

        if "REFERENCES\n" in text.upper():return df

        if not SEEN_ABSTRACT and 'ABSTRACT' not in text.upper(): continue
        else : SEEN_ABSTRACT = True

        if (mostly_numbers(text) or starts_with_figure_or_table(text) or text.startswith('*')
            or ignore_footnote(text)):
          #print(f"<<<WiLL IGNORE: {text}>>>")
          continue
        elif (text[-1] == '-' or text[-1]==':' or len(text)<10
              or not ends_with_period_or_number(text)):
          buffer += text+" "
        else:
          if buffer:# and not ignore_footnote(text):
            text = buffer + " "+ text
            buffer = ''
          if len(df) and df['abstract'].iloc[-1].endswith('-'): df['abstract'].iloc[-1] += text
          elif len(text)>50: df = df.append({'abstract':text.strip()}, ignore_index=True)
  return df
df=parse_paragraphs()

In [69]:
df

,abstract
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...
1,"INTRODUCTION\n Many applications in natural language processing rely on adapting one large-scale, pre-trained language model to multiple downstream applications. Such adaptation is usually done v..."
2,"Many sought to mitigate this by adapting only some parameters or\nlearning external modules for new tasks. This way, we only need\nto store and load a small number of task-speciﬁc parameters in ad..."
3,"often introduce inference latency (Houlsby et al., 2019; Rebufﬁ et al., 2017) by extending model\ndepth or reduce the model’s usable sequence length (Li & Liang, 2021; Lester et al., 2021; Hambard..."
4,We take inspiration from Li et al. (2018a); Aghajanyan et al. (2020) which show that the learned\nover-parametrized models in fact reside on a low intrinsic dimension. We hypothesize that the\ncha...
...,...
58,"To answer these questions, we project W onto the r-dimensional subspace of ∆W by computing U (cid:62)W V (cid:62), with U /V being the left/right singular-vector matrix of ∆W . Then, we compare th..."
59,∆Wq Wq\n Random ∆Wq Wq\n Random\n ||U (cid:62)WqV (cid:62)||F = 0.32\n||Wq||F = 61.95
60,"We draw several conclusions from Table 7. First, ∆W has a stronger correlation with W compared\nto a random matrix, indicating that ∆W ampliﬁes some features that are already in W . Second,\ninste..."
61,8 CONCLUSION AND FUTURE WORK\n Fine-tuning enormous language models is prohibitively expensive in terms of the hardware required\nand the storage/switching cost for hosting independent instances ...


In [72]:
from lmqg import TransformersQG
MODEL_ID = 'lmqg/t5-base-squad-qag'
LLM_QA = TransformersQG(MODEL_ID)

In [76]:
from transformers import AutoTokenizer
TOKENIZER_QA = AutoTokenizer.from_pretrained(MODEL_ID)

In [86]:

def generate_qa_with_sliding_window(text, model, tokenizer, window_size, overlap):
    # Tokenize the text into tokens
    tokens = tokenizer.tokenize(text)

    # Create a sliding window over the tokens
    windows = [tokens[i:i+window_size] for i in range(0, len(tokens), window_size-overlap)]

    # Apply the model to each window and concatenate the results
    results = []
    for window in windows:
        window_text = tokenizer.convert_tokens_to_string(window)
        qa_list = model.generate_qa(window_text)
        results.extend(qa_list)
    return results

# # Initialize the tokenizer
# tokenizer = T5Tokenizer.from_pretrained('t5-base')

# # Define window size and overlap
# window_size = 512
# overlap = 50

# Apply the function to the 'abstract' column
df['qa'] = df['abstract'].apply(lambda x: generate_qa_with_sliding_window(x, LLM_QA, TOKENIZER_QA, window_size=500, overlap=50)) #allow room for special tokens

100%|██████████| 1/1 [00:00<00:00, 849.22it/s]


In [87]:
df[['abstract','qa']].head()

,abstract,qa
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,"[(What is an important paradigm of natural language processing?, large-scale pretraining on general domain data and adaptation to particular tasks or domains), (What becomes less feasible as we pr..."
1,"INTRODUCTION\n Many applications in natural language processing rely on adapting one large-scale, pre-trained language model to multiple downstream applications. Such adaptation is usually done v...","[(What is the major downside of fine-tuning?, the new model contains as many parameters as in the original model), (What is the major downside of fine-tuning?, the new model contains as many param..."
2,"Many sought to mitigate this by adapting only some parameters or\nlearning external modules for new tasks. This way, we only need\nto store and load a small number of task-speciﬁc parameters in ad...","[(What did many seek to mitigate by adapting only some parameters or learning external modules for new tasks?, adapting only some parameters or learning external modules), (What did this way, we o..."
3,"often introduce inference latency (Houlsby et al., 2019; Rebufﬁ et al., 2017) by extending model\ndepth or reduce the model’s usable sequence length (Li & Liang, 2021; Lester et al., 2021; Hambard...","[(What does inference latency introduce?, by extending model depth or reduce the model’s usable sequence length), (What is a trade-off between efficiency and model quality?, these method often fai..."
4,We take inspiration from Li et al. (2018a); Aghajanyan et al. (2020) which show that the learned\nover-parametrized models in fact reside on a low intrinsic dimension. We hypothesize that the\ncha...,"[(What do we take inspiration from?, Li et al. (2018a); Aghajanyan et al. (2020)), (What do we hypothesize that the change in weights during model adaptation also has?, a low “intrinsic rank”), (W..."


In [88]:
df=df.explode('qa')
df.head(100)

,abstract,qa
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,"(What is an important paradigm of natural language processing?, large-scale pretraining on general domain data and adaptation to particular tasks or domains)"
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,"(What becomes less feasible as we pre-train larger models?, full fine-tuning)"
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,"(What does LoRA do?, freezes the pretrained model weights and injects trainable rank decomposition matrices into each layer of the Transformer architecture)"
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,"(How many times can LoRA reduce the number of trainable parameters?, 10,000 times)"
1,"INTRODUCTION\n Many applications in natural language processing rely on adapting one large-scale, pre-trained language model to multiple downstream applications. Such adaptation is usually done v...","(What is the major downside of fine-tuning?, the new model contains as many parameters as in the original model)"
...,...,...
22,"4.2 APPLYING LORA TO TRANSFORMER\n In principle, we can apply LoRA to any subset of weight matrices in a neural network to reduce the\nnumber of trainable parameters. In the Transformer architect...","(How many weight matrices are in the MLP module?, two)"
22,"4.2 APPLYING LORA TO TRANSFORMER\n In principle, we can apply LoRA to any subset of weight matrices in a neural network to reduce the\nnumber of trainable parameters. In the Transformer architect...","(How many weight matrices are in the self-attention module?, four)"
22,"4.2 APPLYING LORA TO TRANSFORMER\n In principle, we can apply LoRA to any subset of weight matrices in a neural network to reduce the\nnumber of trainable parameters. In the Transformer architect...","(What do we treat Wq as?, a single matrix of dimension dmodel dmodel)"
22,"4.2 APPLYING LORA TO TRANSFORMER\n In principle, we can apply LoRA to any subset of weight matrices in a neural network to reduce the\nnumber of trainable parameters. In the Transformer architect...","(What do we limit our study to?, adapting the attention weights for downstream tasks)"


In [89]:
df['question']=df['qa'].apply(lambda x:x[0])
df['answer']=df['qa'].apply(lambda x:x[1])
df.drop(['qa'],axis=1,inplace=True)

In [93]:
df=df.drop_duplicates()
df

,abstract,question,answer
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,What is an important paradigm of natural language processing?,large-scale pretraining on general domain data and adaptation to particular tasks or domains
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,What becomes less feasible as we pre-train larger models?,full fine-tuning
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,What does LoRA do?,freezes the pretrained model weights and injects trainable rank decomposition matrices into each layer of the Transformer architecture
0,ABSTRACT\n An important paradigm of natural language processing consists of large-scale pretraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger m...,How many times can LoRA reduce the number of trainable parameters?,"10,000 times"
1,"INTRODUCTION\n Many applications in natural language processing rely on adapting one large-scale, pre-trained language model to multiple downstream applications. Such adaptation is usually done v...",What is the major downside of fine-tuning?,the new model contains as many parameters as in the original model
...,...,...,...
62,"There are many directions for future works. 1) LoRA can be combined with other efﬁcient adaptation methods, potentially providing orthogonal improvement. 2) The mechanism behind ﬁne-tuning\nor LoR...",What can LoRA be combined with?,other efficient adaptation methods
62,"There are many directions for future works. 1) LoRA can be combined with other efﬁcient adaptation methods, potentially providing orthogonal improvement. 2) The mechanism behind ﬁne-tuning\nor LoR...",What is far from clear?,The mechanism behind fine-tuning or LoRA
62,"There are many directions for future works. 1) LoRA can be combined with other efﬁcient adaptation methods, potentially providing orthogonal improvement. 2) The mechanism behind ﬁne-tuning\nor LoR...",What do we believe makes it more tractable than full fine-tuning?,LoRA
62,"There are many directions for future works. 1) LoRA can be combined with other efﬁcient adaptation methods, potentially providing orthogonal improvement. 2) The mechanism behind ﬁne-tuning\nor LoR...",What do we mostly depend on to select the weight matrices to apply LoRA to?,heuristics


In [100]:
df.to_csv(data_dir+'/'+'LORA.csv',index=False)